In [13]:
import pandas as pd
train = pd.read_csv("training_data.csv")
test = pd.read_csv("testing_data.csv")
data = pd.concat([train, test], ignore_index=True)
data

,tweet,sarcasm,sentiment,dialect
0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa
1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa
2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa
3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf
4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa
...,...,...,...,...
15543,تربكني الذكرى ليا مر طاريهوانسى البشر حولي وال...,False,NEU,msa
15544,وانا احسبهم الحين مايتركون حركاتهم,False,NEG,msa
15545,#فههههههههههد_غشششششششاااااااام_الببببببصصصصصم...,False,NEU,msa
15546,- لــو كــان الامـــر بيدي لأخفيت انهيار دموعي...,False,NEG,msa


In [14]:
X = data["tweet"]
y = data["sentiment"]

data = pd.DataFrame(data, columns=["tweet", "sentiment"])
print(data["tweet"].value_counts()) 

tweet
"RT @M__albugaily: قردوغان حليف اسرائيل يدعم تنظيم داعش في سوريا ومع ذلك لم يغضب عليه أحد ويهدده كما يفعل مع مصر التي تواجه الإرهاب بشكل عام…"    3
"RT @ahlalsunna2: #سوريا :الجيش السوري الحر يسيطر على قرى براغيدة والشيخ ريح والبل وجارز بريف حلب الشمالي بعد معارك مع تنظيم داعش"                3
"RT @ajmubasher: وزير خارجية #بريطانيا يدعو للتظاهر ضد #روسيا#حلب #سورياhttps://t.co/0eBxBmcxua"                                                  2
"RT @al8nas_ksa: نائب العام لجماعة #الإخوان_المسلمين في #سوريا ..#إيران تريد تسليم سوريا للإخوان كونهم الأقرب لهم . #يا_إخوان #أردوغان h…"        2
الصحافة أحاديث مع أناس يحسنون الكلام ننشرها علي أناس لايحسنون القراءة. أنيس منصور                                                                 2
                                                                                                                                                 ..
كان الأولى أن يكتبوا على جميع حوائط المدينة: حافظوا على نظافة نفسيتكم                                     

In [15]:
print(data['sentiment'][12]) # la classe qui est dans la ligne 12
print(data['tweet'][12]) # le tweet qui est dans la ligne 12
data["tweet"][225] # le tweet qui est dans la ligne 225

NEG
اف هذا🤢


'فيه ناس بتبعتلي رسائل عتاب رقيقة جداً والله يا جماعة أنا ما عنديش أي نية أكون كاهن ولا ملاك وبقول للخروف يا خروف في وشه ومش هتغير'

In [16]:
import warnings
from aaransia import transliterate, SourceLanguageError

warnings.simplefilter('ignore')

for i in range(len(data)):
    row = data.iloc[i]["tweet"]
    try:
        row = transliterate(row, source='ar', target='ar', universal=True)
        data.loc[:, "tweet"][i] = row
    except SourceLanguageError as e:
        print(f"Erreur de translittération à l'index {i}: {e}")

# Affichage d'un exemple aprés le premier prétraitement
data["tweet"][225]

'فيه ناس بتبعتلي رسايل عتاب رقيقة جدا واله يا جماعة انا ما عنديش اي نية اكون كاهن ولا ملاك وبقول لخروف يا خروف في وشه ومش هتغير'

In [17]:
 import re

# Normalisation des données
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

# Applique la normalisation aux tweets
data['tweet'] = data['tweet'].apply(normalize_arabic)

print(data['tweet'][12])

اف هذا🤢


In [18]:
import re
print("Exemples avant faire le nettoyage :")
print(data['tweet'][1])
print(data['tweet'][2])
print(data['tweet'][3])
print(data['tweet'][12])

# Applique les modifications

# Supprimer les hyperliens
data['tweet'] = [re.sub(r'http\S+ | htps\S+', '', str(s)) for s in data['tweet']]

# Supprimer les URL
data['tweet'] = [re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', '', str(s)) for s in data['tweet']]

# Supprimer les mots commençant par @
data['tweet'] = [re.sub(r'@\S+', '', str(s)) for s in data['tweet']]

# Supprimer # et _
data['tweet'] = data['tweet'].str.replace("_", " ").str.replace("#", "")

data['tweet'] = data['tweet'].str.replace('. | , | ، | ؛', ' ')

# Supprimer les mots réservés sur Twitter
data['tweet'] = [re.sub(r'\bRT\b | \bRetweeted\b', '', str(s)) for s in data['tweet']]

data['tweet'] = [re.sub(r'[\u0660-\u0669\u06F0-\u06F9]+', '', str(s)) for s in data['tweet']]

# Supprimer les caractères consécutifs en double
data['tweet'] = [re.sub(r"(.)\1{2,}", r"\1", str(s)) for s in data['tweet']] if len(data['tweet']) > 2 else data['tweet'][:2]

data['tweet'] = data['tweet'].str.replace('\d+', ' ')
data['tweet'] = data['tweet'].str.replace('\n', ' ')
data['tweet'] = data['tweet'].str.replace('/', ' ')
data['tweet'] = [re.sub(r'[^\w\s]', '', str(s)) for s in data['tweet']]

# Remplacer les valeurs nulles par une chaîne de caractères vide
data['tweet'] = data['tweet'].fillna('')

# Vérifie un exemple
print("\nExemples après faire le nettoyage :")
print(data['tweet'][1])
print(data['tweet'][2])
print(data['tweet'][3])

Exemples avant faire le nettoyage :
"مع فيدر يا اجا والكبار 😍 htps:/t.co/hrBeHbkBNu"
“الداعون لمبدا الاختلاط بين الجنسين؛ كالداعين لالغاء التسعيره كلاهما يريد تصفيه السوق السوداء بجعلها حره.” #الاختلاط
"@ihe_94 @ya78m @amoo5 @badiajnikhar @Oukasafa @reoshalm @Mnory202 مساكين من الصبح و هوما رايحين راجعين عاا غوغل تعبت بدالهم ههه ي"
اف هذا🤢

Exemples après faire le nettoyage :
مع فيدر يا اجا والكبار 
الداعون لمبدا الاختلاط بين الجنسين كالداعين لالغاء التسعيره كلاهما يريد تصفيه السوق السوداء بجعلها حره الاختلاط
 مساكين من الصبح و هوما رايحين راجعين عاا غوغل تعبت بدالهم ه ي


In [19]:
import nltk
from nltk.corpus import stopwords
# Affiche quelques mots (stop-words)
print(stopwords.words('arabic'))

stopwords = list(set(nltk.corpus.stopwords.words('arabic')))
data["tweet"] = data["tweet"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

# Affichage un exemple apés faire le nettoyage des données (stop-words)
print("\nExemples aprés faire le nettoyage des stop-words:")
print(data['tweet'][1])
print(data['tweet'][2])
print(data['tweet'][3])
print(data['tweet'][12])

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [20]:
from tashaphyne.stemming import ArabicLightStemmer

# Stemmer_LIGHT : Supprimer "suffixes" et "affixes" 
ArListem = ArabicLightStemmer()
def stemmer_light(text):
    text_words = []
    words = text.split(" ")
    for c in words:
        stem = ArListem.light_stem(c)
        text_words.append(stem)
    return ' '.join(text_words)

# Root Stemming : Transformer le mot dans sa forme racine
def stemmer_root(text):
    text_words = []
    words = text.split(" ")
    for c in words:
        stem = ArListem.light_stem(c)
        text_words.append(stem)
    return ' '.join(text_words)

sentences = [stemmer_light(text) for text in data['tweet']]

data['tweet'] = sentences
print(data['tweet'][12])

اف


In [21]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in  vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# Nous inspectons le mot principal dans un échantillon de notre corpus sans dériver les mots
from pprint import pprint

topWords=get_top_n_words(data['tweet'],n=20)
pprint(topWords)

[('عل', 2710),
 ('ان', 2470),
 ('لا', 1311),
 ('له', 1291),
 ('مصر', 1098),
 ('tco', 1067),
 ('لي', 1032),
 ('نا', 853),
 ('نت', 799),
 ('على', 757),
 ('حد', 644),
 ('حلب', 599),
 ('فا', 577),
 ('كل', 553),
 ('قول', 547),
 ('مش', 541),
 ('رييس', 527),
 ('دول', 526),
 ('او', 483),
 ('نه', 469)]


In [25]:
import nltk
import arabic_reshaper

arab_stopwords = list(set(nltk.corpus.stopwords.words("arabic")))
sample_corpus=' '.join(data['tweet'])
data_arb = arabic_reshaper.reshape(sample_corpus)

ModuleNotFoundError: No module named 'arabic_reshaper'

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
all_datasets= []

print(data["sentiment"].value_counts())
sentiment = list(data["sentiment"].unique())
print(sentiment)

vectorizer = CountVectorizer()
vectorizer.fit(data["tweet"])
vocab_size = len(vectorizer.vocabulary_)
print(f"Il y a {vocab_size} mots différents dans notre corpus.")

In [ ]:
import nltk

from nltk.tokenize import word_tokenize
tokenizedWords = []
documents = []

# Chaque document contient tuple ==> la liste de mot et categorie
for i in data.index:
    sentiment = data["sentiment"][i]
    review = data["tweet"][i]
    tokenizedWord = word_tokenize(review)
    document = [tokenizedWord, sentiment]
    documents.append(document)

# Chaque element de documents contient une liste = [contient tous les mots de document, son categorie]
sentiment
print("Taille documents : ", len(documents))
data["tweet"][4]
# Chaque élément de documents contient une liste qui contient tous les mots présents dans ce document et son catégorie (sentiment soit 0 ou bien 1) comme par exemple ce commentaire 
documents[4]

In [ ]:
print(data['tweet'][12])

listeall=[]

for i in data["tweet"]:
    review = i
    tokenizedWord = word_tokenize(review)
    for j in tokenizedWord:
        listeall.append(j)
# listeall est une liste qui contient tous les mots de tous les documents , qu'on va utiliser par la suite dans la partie de bag of word 
print(len(listeall))

In [ ]:
all_words = nltk.FreqDist(w.lower() for w in listeall)
word_features = list(all_words)[:2000]
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

df_fdist = pd.DataFrame.from_dict(all_words, orient='index')
df_fdist.columns = ['Frequency']
df_fdist.index.name = 'Term'
print(df_fdist)

In [ ]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]

# Former le classificateur Naive Bayes
model = nltk.NaiveBayesClassifier.train(train_set)

# Tester le modèle
print("Accuracy avec le model Naive Bayes : ", nltk.classify.accuracy(model, test_set), "%")
test_set[0][0]
model.classify(test_set[0][0])

In [ ]:
test_result = []
gold_result = []

for i in range(len(test_set)):
    test_result.append(model.classify(test_set[i][0]))
    gold_result.append(test_set[i][1])

CM = nltk.ConfusionMatrix(gold_result, test_result)
print(CM)

In [ ]:
model.show_most_informative_features(5)

In [ ]:
import nltk
import random
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Vectorisation des données textuelles
vectorizer = CountVectorizer(max_features = 8000)
X = data["tweet"]
y = data["sentiment"]

# Séparation des données en 2 parties : entraînement / test
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

train_X = vectorizer.fit_transform(train_X)
test_X = vectorizer.transform(test_X)

# Entraînement du modèle
modelMulti = MultinomialNB()
modelMulti.fit(train_X, train_y)

# Test du classificateur
predictions = modelMulti.predict(test_X)

# Calcul des métriques de performance
score_NVB = accuracy_score(test_y, predictions) * 100
precision_NVB = precision_score(test_y, predictions, average = None) * 100
recall_NVB = recall_score(test_y, predictions, average = None) * 100

# Affichage des résultats
print('Accuracy du modèle sur l\'ensemble de test : ', round(score_NVB, 2), '%')
report = classification_report(test_y, predictions)
print(report) 

In [ ]:
for N in range(1,10):
    vectorizer_G = CountVectorizer(max_features=8000, ngram_range=(1,N))

    X = data["tweet"]
    y = data["sentiment"]

    # Séparation des données en 2 parties : entraînement / test
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2,random_state=42)
 
    train_X = vectorizer_G.fit_transform(train_X)
    test_X = vectorizer_G.transform(test_X)
    
    # Entraînement du modèle
    modelMulti_G = MultinomialNB()
    modelMulti_G.fit(train_X, train_y)
    
    # Test du classificateur
    predictions_2G = modelMulti_G.predict(test_X)
    print('Accuracy du modèle avec {} gramme: '.format(N), round(accuracy_score(test_y, predictions_2G) * 100, 2), '%')

In [ ]:
from sklearn.naive_bayes import  MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

vectorizer_G = CountVectorizer(max_features=8000, ngram_range=(1,3))

X = data["tweet"]
y = data["sentiment"]

# Séparation des données en 2 parties : entraînement / test
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2,random_state=42)
 
train_X = vectorizer_G.fit_transform(train_X) 
test_X = vectorizer_G.transform(test_X)

# Entraînement du modèle
modelMulti_G = MultinomialNB()
modelMulti_G.fit(train_X, train_y)

# Test the classifier
predictions_G = modelMulti_G.predict(test_X)
score_NVB_G = accuracy_score(test_y, predictions_G) * 100
precision_NVB_G = precision_score(test_y, predictions_G, average = None) * 100
recall_NVB_G = recall_score(test_y, predictions_G, average = None) * 100
print('Accuracy du modèle sur l\'ensemble de test : ', round(accuracy_score(test_y, predictions_G) * 100, 2), '%')
report = classification_report(test_y, predictions_G)
print(report)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Vectorisation des critiques
vectorizer = CountVectorizer(max_features=8000)


# Séparation des données en 2 parties : entraînement / test
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2,random_state=42)
 
train_X = vectorizer.fit_transform(train_X)
test_X = vectorizer.transform(test_X)

# Entraînement du modèle
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=20, min_samples_split=2, min_samples_leaf=1)
decision_tree = decision_tree.fit(train_X, train_y)
predictions_tree = decision_tree.predict(test_X)
score_DT = accuracy_score(test_y, predictions_tree) * 100
precision_DT = precision_score(test_y, predictions_tree, average = None) * 100
recall_DT = recall_score(test_y, predictions_tree, average = None) * 100
print('Accuracy du modèle decision_tree avec CountVectorizer sur l\'ensemble de test : ', round(accuracy_score(test_y, predictions_tree) * 100, 2), '%')

report = classification_report(test_y, predictions_tree)
print(report)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Vectorisation des critiques
vectorizer = CountVectorizer(max_features = 8000)


# Séparation des données en 2 parties : entraînement / test
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2,random_state=42)

train_X = vectorizer.fit_transform(train_X)
test_X = vectorizer.transform(test_X)

# Entraînement du modèle
rf = RandomForestClassifier(max_depth = None, random_state = 32, n_estimators = 1000)
rf = rf.fit(train_X, train_y)
predictions = rf.predict(test_X)
score_RF = accuracy_score(test_y, predictions) * 100
precision_RF = precision_score(test_y, predictions, average = None) * 100
recall_RF = recall_score(test_y, predictions, average = None) * 100
print('Accuracy du modèle Random_Forest avec CountVectorizer sur l\'ensemble de test : ', round(accuracy_score(test_y, predictions) * 100, 2), '%')
report = classification_report(test_y, predictions)
print(report)  

In [ ]:
import nltk
import random
from sklearn.naive_bayes import  MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

vectorizer_TFIDF = TfidfVectorizer(max_features=8000)

# Séparation des données en 2 parties : entraînement / test
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2,random_state=42)
 
train_X = vectorizer_TFIDF.fit_transform(train_X) 
test_X = vectorizer_TFIDF.transform(test_X)

# Entraînement du modèle
modelMulti_tf = MultinomialNB()
modelMulti_tf.fit(train_X, train_y)

# Test the classifier
predictions_TF = modelMulti_tf.predict(test_X)
score_NVB_TF = accuracy_score(test_y, predictions_TF) * 100
precision_NVB_TF = precision_score(test_y, predictions_TF, average = None) * 100
recall_NVB_TF= recall_score(test_y, predictions_TF, average = None) * 100
print('Accuracy du modèle MultinomialNB avec TfidfVectorizer et sans n-grames sur l\'ensemble de test : ', round(accuracy_score(test_y, predictions_TF) * 100, 2), '%')
report = classification_report(test_y, predictions_TF)
print(report)

In [ ]:
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Modèle 01
data.columns = ["tweet", "sentiment"]

# Vectorisation des critiques
vectorizer = CountVectorizer(max_features = 8000, ngram_range=(1,3))
X = vectorizer.fit_transform(data["tweet"])
y = data["sentiment"]

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=42)
lin_clf = svm.LinearSVC()
lin_cl = lin_clf.fit(train_X, train_y)
predictions = lin_cl.predict(val_X)
score_svm = accuracy_score(val_y, predictions) * 100
precision_svm = precision_score(val_y, predictions, average=None) * 100
recall_svm = recall_score(val_y, predictions, average=None) * 100

print('Accuracy du modèle SVM avec CountVectorizer et n-grames sur l\'ensemble de test : ', round(score_svm, 2), '%')
report = classification_report(val_y, predictions)
print(report)

In [ ]:
import pandas as pd
import numpy as np

df_data = np.array([
    [score_NVB, precision_NVB[0], recall_NVB[0], precision_NVB[1], recall_NVB[1], precision_NVB[2], recall_NVB[2]],
    [score_DT, precision_DT[0], recall_DT[0], precision_DT[1], recall_DT[1], precision_DT[2], recall_DT[2]],
    [score_RF, precision_RF[0], recall_RF[0], precision_RF[1], recall_RF[1], precision_RF[2], recall_RF[2]],
    [score_svm, precision_svm[0], recall_svm[0], precision_svm[1], recall_svm[1], precision_svm[2], recall_svm[2]]
])

# Créer un DataFrame pandas avec les colonnes appropriées
columns = ['accuracy', 'precision_NEG', 'recall_NEG', 'precision_NEU', 'recall_NEU', 'precision_POS', 'recall_POS']
index = ['Naive Bayes', 'Decision Tree', 'Random Forest', 'SVM']

df = pd.DataFrame(df_data, columns=columns, index=index)

# Afficher le DataFrame
df

In [ ]:
import pandas as pd
import numpy as np

df_data = np.array([
    [score_NVB, precision_NVB[0], recall_NVB[0], precision_NVB[1], recall_NVB[1], precision_NVB[2], recall_NVB[2]],
    [score_NVB_G, precision_NVB_G[0], recall_NVB_G[0], precision_NVB_G[1], recall_NVB_G[1], precision_NVB_G[2], recall_NVB_G[2]],
    [score_NVB_TF, precision_NVB_TF[0], recall_NVB_TF[0], precision_NVB_TF[1], recall_NVB_TF[1], precision_NVB_TF[2], recall_NVB_TF[2]],
    [score_NVB_TF_G, precision_NVB_TF_G[0], recall_NVB_TF_G[0], precision_NVB_TF_G[1], recall_NVB_TF_G[1], precision_NVB_TF_G[2], recall_NVB_TF_G[2]]
])

# Créer un DataFrame pandas avec les colonnes appropriées
columns = ['accuracy', 'precision_NEG', 'recall_NEG', 'precision_NEU', 'recall_NEU', 'precision_POS', 'recall_POS']
index = ['Naive Bayes avec CountVectorizer 1_gram', 'Naive Bayes avec CountVectorizer 3_gram ',
         'Naive Bayes avec TfidfVectorizer 1_gram', 'Naive Bayes avec TfidfVectorizer 3_gram']

df = pd.DataFrame(df_data, columns=columns, index=index)

# Afficher le DataFrame
df